In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("sample").master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [4]:
cnts_prod_mst_20210205 = spark.read.format("csv").option("header","true").load("C:/Users/b2en/Downloads/Spark 실습/homework_2/cnts_prod_mst_20210205.csv")
cnts_prod_mst_20210206 = spark.read.format("csv").option("header","true").load("C:/Users/b2en/Downloads/Spark 실습/homework_2/cnts_prod_mst_20210206.csv")
cnts_prod_mst_20210207 = spark.read.format("csv").option("header","true").load("C:/Users/b2en/Downloads/Spark 실습/homework_2/cnts_prod_mst_20210207.csv")
cnts_prod_mst_20210208 = spark.read.format("csv").option("header","true").load("C:/Users/b2en/Downloads/Spark 실습/homework_2/cnts_prod_mst_20210208.csv")
vod_wat_day_20210206 = spark.read.format("csv").option("header","true").load("C:/Users/b2en/Downloads/Spark 실습/homework_2/vod_wat_day_20210206.csv")
vod_wat_day_20210207 = spark.read.format("csv").option("header","true").load("C:/Users/b2en/Downloads/Spark 실습/homework_2/vod_wat_day_20210207.csv")

In [3]:
from pyspark.sql.functions import *
from pyspark.sql.window import *

In [5]:
cnts_prod_mst_20210205.createOrReplaceTempView("cnts_prod_mst_20210205")

In [11]:
cnts_prod_mst_20210206.createOrReplaceTempView("cnts_prod_mst_20210206")

In [6]:
spark.sql("SELECT * FROM cnts_prod_mst_20210205")

STRD_DT,CNTS_PROD_ID,CNTS_PROD_NM,CNTS_PROD_TYP_CD,PROD_PRC,SALE_PRC,USE_YN
20210205,PM50773595,런닝맨_21.01.17.일_SBS,10,1500,1500,Y


In [7]:
vod_wat_day_20210206.createOrReplaceTempView("vod_wat_day_20210206")

In [8]:
spark.sql("SELECT * FROM vod_wat_day_20210206")

STRD_DT,SVC_MGMT_NUM,CNTS_PROD_ID,WAT_CNT
20210206,7000000001,PM50773595,1
20210206,7000000002,PM50773595,1
20210206,7000000003,PM50773595,2


In [10]:
spark.sql("SHOW DATABASES")
spark.sql("SHOW TABLES")

database,tableName,isTemporary
,cnts_prod_mst_202...,true
,vod_wat_day_20210206,true


In [14]:
spark.sql("SELECT B.STRD_DT\
                 ,B.SVC_MGMT_NUM\
                 ,B.CNTS_PROD_ID\
                 ,A.CNTS_PROD_NM\
                 ,B.WAT_CNT\
                 ,CASE WHEN A.SALE_PRC >= 0 THEN 'Y' ELSE 'N' END as PYFR_YN\
                 ,A.SALE_PRC\
             FROM cnts_prod_mst_20210206 as A\
             INNER JOIN vod_wat_day_20210206 B\
             ON A.cnts_prod_id = B.cnts_prod_id")

STRD_DT,SVC_MGMT_NUM,CNTS_PROD_ID,CNTS_PROD_NM,WAT_CNT,PYFR_YN,SALE_PRC
20210206,7000000001,PM50773595,런닝맨_21.01.17.일_SBS,1,Y,1500
20210206,7000000002,PM50773595,런닝맨_21.01.17.일_SBS,1,Y,1500
20210206,7000000003,PM50773595,런닝맨_21.01.17.일_SBS,2,Y,1500
